In [2]:
import pandas as pd
import gripql
from tqdm import tqdm

In [3]:
conn = gripql.Connection(url='https://bmeg.io/grip', credential_file='../bmeg_credentials.json')

In [4]:
G = conn.graph("rc6")

In [5]:
projects = G.query().V('Program:TCGA').out("projects").render("$._gid").execute()

In [6]:
dfs = {}

for p in tqdm(projects):
    q = G.query().V(p).out("cases").out("samples").as_("sample")
    q = q.has(gripql.eq("$.gdc_attributes.sample_type", "Primary Tumor"))
    q = q.out("aliquots").out("gene_expressions")
    q = q.render(["$sample.submitter_id", "$.values"])
    
    data = {}
    for row in q:
        data[row[0]] = row[1]
    dfs[p] = pd.DataFrame(data).transpose()

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 33/33 [30:34<00:00, 55.60s/it]


In [7]:
mergeDF = pd.concat( dfs.values() )

In [8]:
ensembl2hugo = pd.read_csv("./ensembl2hugo.tsv", sep="\t", header=None, index_col=0)[1]

FileNotFoundError: [Errno 2] No such file or directory: './ensembl2hugo.tsv'

In [9]:
hugoDF = mergeDF.rename(columns=ensembl2hugo)

NameError: name 'ensembl2hugo' is not defined

In [10]:
hugoDF.to_csv("tcga.tumor.hugo.tsv", sep="\t")

NameError: name 'hugoDF' is not defined

In [11]:
q = G.query().V(projects[0]).out("cases").out("samples").as_("sample")
#q = q.has(gripql.eq("$.gdc_attributes.sample_type", "Primary Tumor"))
q = q.distinct("$.gdc_attributes.sample_type")
q.execute()

[{'gid': 'Sample:3f2992eb-bdc0-442d-a34b-6c042ea87f6b',
  'label': 'Sample',
  'data': {'gdc_attributes': {'sample_type': 'Recurrent Tumor',
    'submitter_id': 'TCGA-G5-6572-02A',
    'tumor_descriptor': None},
   'project_id': 'Project:TCGA-READ',
   'sample_id': '3f2992eb-bdc0-442d-a34b-6c042ea87f6b',
   'submitter_id': 'TCGA-G5-6572-02A'}},
 {'gid': 'Sample:c913e113-9731-488a-ac17-652f5a52f771',
  'label': 'Sample',
  'data': {'gdc_attributes': {'sample_type': 'Solid Tissue Normal',
    'submitter_id': 'TCGA-AG-A01W-11A',
    'tumor_descriptor': None},
   'project_id': 'Project:TCGA-READ',
   'sample_id': 'c913e113-9731-488a-ac17-652f5a52f771',
   'submitter_id': 'TCGA-AG-A01W-11A'}},
 {'gid': 'Sample:bb4fb9dc-e786-4fcf-8a1c-f5791bc900c3',
  'label': 'Sample',
  'data': {'gdc_attributes': {'sample_type': 'Blood Derived Normal',
    'submitter_id': 'TCGA-AG-A00C-10A',
    'tumor_descriptor': None},
   'project_id': 'Project:TCGA-READ',
   'sample_id': 'bb4fb9dc-e786-4fcf-8a1c-f5791b

In [12]:
normdfs = {}

for p in tqdm(projects):
    q = G.query().V(p).out("cases").out("samples").as_("sample")
    q = q.has(gripql.eq("$.gdc_attributes.sample_type", "Solid Tissue Normal"))
    q = q.out("aliquots").out("gene_expressions")
    q = q.render(["$sample.submitter_id", "$.values"])
    
    data = {}
    for row in q:
        data[row[0]] = row[1]
    normdfs[p] = pd.DataFrame(data).transpose()

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 33/33 [02:50<00:00,  5.16s/it]


In [13]:
normMergeDF = pd.concat( normdfs.values() )
normHugoDF = normMergeDF.rename(columns=ensembl2hugo)

NameError: name 'ensembl2hugo' is not defined

In [ ]:
normHugoDF.to_csv("tcga.normal.hugo.tsv", sep="\t")